In [17]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [18]:
config = {
    'user': 'mehdi',
    'password': 'Sz2295780211',
    'host': 'localhost',
    'database': 'data',
    'auth_plugin' : 'mysql_native_password'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

# Query 1

In [44]:
cursor.execute("""SELECT EXTRACT(YEAR FROM c.collision_date) as year, COUNT(*) as number_of_collisions
FROM Collisions c 
GROUP BY EXTRACT(YEAR FROM c.collision_date)
LIMIT 20""")

In [45]:
res = cursor.fetchall()

In [46]:
for x in res:
    print(x)

(2002, 544741)
(2003, 538954)
(2001, 522562)
(2004, 538295)
(2005, 532725)
(2006, 498849)
(2007, 501908)
(2017, 7)
(2018, 21)


In [50]:
print(markdown_table(res, ["Year", "Count"]))

Year | Count
:---: | :---:
2002 | 544741
2003 | 538954
2001 | 522562
2004 | 538295
2005 | 532725
2006 | 498849
2007 | 501908
2017 | 7
2018 | 21



# Query 2

In [53]:
cursor.execute("""CREATE VIEW MostPopularVehicle AS
(SELECT COUNT(p.id) as number_of_vehicles, v.vehicle_make as brand_id
			     FROM Parties p, Vehicles v
			     WHERE p.id = v.party_id GROUP BY v.vehicle_make
			     ORDER BY number_of_vehicles desc
			     limit 1)""")

ProgrammingError: 1050 (42S01): Table 'MostPopularVehicle' already exists

In [52]:
res = cursor.fetchall()
for x in res:
    print(x)

InterfaceError: No result set to fetch from.

In [17]:
cursor.execute("""SELECT vm.description as most_popular, vp.number_of_vehicles as number_of_vehicles
FROM VehiculeMake vm, MostPopularVehicle vp
WHERE vm.id = 1""")

In [18]:
res = cursor.fetchall()

In [ ]:
for x in res:
    print(x)

In [19]:
print(markdown_table(res, ["Brand", "Count"]))

Brand | Count
:---: | :---:
FORD | 1129719



# Query 3

In [116]:
cursor.execute("""SELECT
  (
    SELECT
      COUNT(*)
    FROM
      (
        SELECT
          c.case_id
        FROM
          Collisions c,
          Factors f,
          Lighting l
        WHERE
          c.case_id = f.case_id
          AND f.lighting = l.id
          AND l.description LIKE 'dark%'
      ) AS DarkCols
  ) / (
    SELECT
      COUNT(*)
    FROM
      Collisions
  )""")

In [117]:
res = cursor.fetchall()
for x in res:
    print(x)

(Decimal('0.2802'),)


# Query 4

In [124]:
cursor.execute("""SELECT COUNT(snowy.case_id)
FROM Collisions snowy , Factors f , Weather w 
WHERE snowy.case_id = f.case_id AND
      (f.weather_1 = w.id OR
      f.weather_2 = w.id) AND
      w.description LIKE 'snowing' """)

In [125]:
res = cursor.fetchall()
for x in res:
    print(x)

(8542,)


# Query 5

In [32]:
cursor.execute("""SELECT DAYOFWEEK(collision_date) as day , COUNT(*) as counts 
FROM   Collisions
GROUP BY day
ORDER BY counts desc
LIMIT 1""")

In [33]:
res = cursor.fetchall()
for x in res:
    print(x)

(6, 614143)


# Query 6

In [8]:
cursor.execute("""SELECT weather, SUM(counts) as counts FROM
(SELECT f.weather_1 as weather, COUNT(*) as counts
FROM Factors f
GROUP BY f.weather_1
UNION ALL
SELECT f.weather_2 as weather, COUNT(*) as counts
FROM Factors f
GROUP BY f.weather_2) AS WeatherList
GROUP BY weather
ORDER BY counts DESC""")

In [9]:
res = cursor.fetchall()

In [ ]:
for x in res:
    print(x)

In [10]:
print(markdown_table(res, ['Weather type', 'Count']))

Weather type | Count
:---: | :---:
None | 3593584
1 | 2942023
2 | 548420
6 | 223797
3 | 21289
7 | 13958
4 | 8542
5 | 6965



# Query 7

In [100]:
cursor.execute("""SELECT COUNT(*) as count 
FROM Parties p, Factors f
WHERE 
	p.at_fault = 1 
	AND 
	p.financial_responsibility IN 
		(SELECT id FROM FinancialResponsibility WHERE description='Y')
	AND 
	f.case_id = p.case_id
	AND 
		(f.road_condition_1 IN
			(SELECT id FROM RoadCondition WHERE description='loose material') 
		OR
		f.road_condition_2 IN 
			(SELECT id FROM RoadCondition WHERE description='loose material') 
		)""")

In [101]:
res = cursor.fetchall()
for x in res:
    print(x)

(4818,)


# Query 8

In [56]:
cursor.execute("""CREATE INDEX index_victim_age ON Victims(victim_age)""")

In [63]:
cursor.execute("""SELECT MEDIAN(victim_age) FROM Victims""")

KeyboardInterrupt: 

In [ ]:
res = cursor.fetchall()
for x in res:
    print(x)

In [3]:
cursor.execute("""SELECT
  victim_seating_position
FROM
  Victims
GROUP BY
  victim_seating_position
ORDER BY
  COUNT(*) DESC
limit
  1""")

In [4]:
res = cursor.fetchall()

In [5]:
for x in res:
    print(x)

(3,)


In [6]:
print(markdown_table(res, ['Fraction']))

NameError: name 'markdown_table' is not defined

# Query 9

In [53]:
cursor.execute("""SELECT
  (
    (
      SELECT
        COUNT(V.case_id)
      FROM
        Victims V,
        VictimSafetyEquipment S
      WHERE
        (
          V.victim_safety_equipment_1 = S.id
          OR V.victim_safety_equipment_2 = S.id
        )
        AND S.description IN ('C', 'E', 'G')
    ) / (
      SELECT
        COUNT(*)
      FROM
        Victims
    )
  )""")

In [54]:
res = cursor.fetchall()

In [55]:
print(markdown_table(res, ['Fraction']))

Fraction
:---:
0.7495



# Query 10

In [37]:
cursor.execute("""SELECT
   CASE WHEN collision_time BETWEEN '00:00:00'
   AND '00:59:00' THEN '0' WHEN collision_time BETWEEN '01:00:00'
   AND '01:59:00' THEN '1' WHEN collision_time BETWEEN '02:00:00'
   AND '02:59:00' THEN '2' WHEN collision_time BETWEEN '03:00:00'
   AND '03:59:00' THEN '3' WHEN collision_time BETWEEN '04:00:00'
   AND '04:59:00' THEN '4' WHEN collision_time BETWEEN '05:00:00'
   AND '05:59:00' THEN '5' WHEN collision_time BETWEEN '06:00:00'
   AND '06:59:00' THEN '6' WHEN collision_time BETWEEN '07:00:00'
   AND '07:59:00' THEN '7' WHEN collision_time BETWEEN '08:00:00'
   AND '08:59:00' THEN '8' WHEN collision_time BETWEEN '09:00:00'
   AND '09:59:00' THEN '9' WHEN collision_time BETWEEN '10:00:00'
   AND '10:59:00' THEN '10' WHEN collision_time BETWEEN '11:00:00'
   AND '11:59:00' THEN '11' WHEN collision_time BETWEEN '12:00:00'
   AND '12:59:00' THEN '12' WHEN collision_time BETWEEN '13:00:00'
   AND '13:59:00' THEN '13' WHEN collision_time BETWEEN '14:00:00'
   AND '14:59:00' THEN '14' WHEN collision_time BETWEEN '15:00:00'
   AND '15:59:00' THEN '15' WHEN collision_time BETWEEN '16:00:00'
   AND '16:59:00' THEN '16' WHEN collision_time BETWEEN '17:00:00'
   AND '17:59:00' THEN '17' WHEN collision_time BETWEEN '18:00:00'
   AND '18:59:00' THEN '18' WHEN collision_time BETWEEN '19:00:00'
   AND '19:59:00' THEN '19' WHEN collision_time BETWEEN '20:00:00'
   AND '20:59:00' THEN '20' WHEN collision_time BETWEEN '21:00:00'
   AND '21:59:00' THEN '21' WHEN collision_time BETWEEN '22:00:00'
   AND '22:59:00' THEN '22' ELSE '23' END AS hour_ranges,
   count(1) / (SELECT COUNT(*) FROM Collisions) as count
 FROM
   Collisions
 GROUP BY
   hour_ranges
 LIMIT 20""")

In [49]:
def markdown_table(res, titles):
    if len(titles) != len(res[0]):
        raise Exception("arg arrays don't match")

    table = ' | '.join(titles)
    table += '\n'
    table += ' | '.join([":---:"] * len(titles))
    table += '\n'

    for row in res:
        table += ' | '.join([str(elem) for elem in row])
        table += '\n'

    return table

In [38]:
res = cursor.fetchall()

In [52]:
print(markdown_table(res, ['Hour Ranges', 'Counts']))

Hour Ranges | Counts
:---: | :---:
15 | 0.0775
19 | 0.0443
7 | 0.0517
11 | 0.0489
17 | 0.0790
16 | 0.0733
8 | 0.0523
6 | 0.0262
12 | 0.0578
23 | 0.0319
22 | 0.0286
10 | 0.0423
2 | 0.0181
14 | 0.0655
1 | 0.0183
9 | 0.0409
18 | 0.0630
13 | 0.0578
21 | 0.0328
20 | 0.0349

